In [56]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [1]:
import cv2
import numpy as np


class RubikScanner:
    """
    Class for scanning a Rubik's cube and extracting the colors of the stickers
    """

    def __init__(self):
        self.face = None
        self.colors = None
        self.faces = {
            "U": None,
            "D": None,
            "L": None,
            "R": None,
            "F": None,
            "B": None,
        }
        self.face_colors = {
            "U": None,
            "D": None,
            "L": None,
            "R": None,
            "F": None,
            "B": None,
        }
        self.current_face_index = 0
        self.face_order = ["U", "D", "L", "R", "F", "B"]
        self.processed_faces = {face: None for face in self.faces.keys()}

    def capture_face(self, camera_id=0):
        """
        Capture the image from the camera
        """
        cap = cv2.VideoCapture(camera_id)

        if not cap.isOpened():
            print("Cannot open camera")
            return

        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                print("Can't receive frame (stream end?). Exiting ...")
                break
            frame = cv2.flip(frame, 1)

            height, width, _ = frame.shape
            rect_top_left = (width // 2 - 150, height // 2 - 150)
            rect_bottom_right = (width // 2 + 150, height // 2 + 150)
            cell_size = 100

            # Vẽ khung hình vuông 3x3
            cv2.rectangle(frame, rect_top_left, rect_bottom_right, (0, 255, 0), 2)

            # Vẽ tâm của mỗi ô
            for i in range(3):
                for j in range(3):
                    center_x = rect_top_left[0] + j * cell_size + cell_size // 2
                    center_y = rect_top_left[1] + i * cell_size + cell_size // 2
                    cv2.circle(frame, (center_x, center_y), 8, (255, 255, 255), -1)

            # Hiển thị các mặt đã lưu trong cửa sổ khác
            self.display_saved_faces()

            cv2.imshow("Press 's' to save, 'n' for next face", frame)

            key = cv2.waitKey(1)
            if key == ord("s"):
                self.save_current_face(frame, height, width)
            elif key == ord("n"):
                if self.faces[self.face_order[self.current_face_index]] is not None:
                    if self.current_face_index < len(self.faces) - 1:
                        self.current_face_index += 1
                    else:
                        print("Đã quét xong tất cả các mặt.")
                        break
                else:
                    print(
                        "Vui lòng lưu mặt hiện tại trước khi chuyển sang mặt tiếp theo."
                    )
            elif key == ord("q"):
                break

        cap.release()
        cv2.destroyAllWindows()

    def display_saved_faces(self):
        """
        Hiển thị các mặt đã lưu trong cửa sổ khác
        """
        display_frame = np.zeros((300, 200, 3), dtype=np.uint8)

        for i, (face_name, face) in enumerate(self.faces.items()):
            if face is not None:
                row = i % 3
                col = i // 3
                y = row * 100
                x = col * 100
                processed_face = self.processed_faces[face_name]
                processed_face_resized = cv2.resize(
                    processed_face, (100, 100)
                )  # Resize the processed face
                display_frame[y : y + 100, x : x + 100] = (
                    processed_face_resized  # Assign the resized processed face
                )
                cv2.putText(
                    display_frame,
                    face_name,
                    (x + 40, y + 90),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,
                    (0, 0, 0),
                    2,
                    cv2.LINE_AA,
                )
                self.draw_grid(display_frame[y : y + 100, x : x + 100])

        cv2.imshow("Saved Faces", display_frame)

    def draw_grid(self, face):
        """
        Vẽ khung hình 3x3 cho mặt Rubik
        """
        cell_size = face.shape[0] // 3
        for i in range(1, 3):
            # Vẽ các đường ngang
            cv2.line(
                face, (0, i * cell_size), (face.shape[1], i * cell_size), (0, 0, 0), 1
            )
            # Vẽ các đường dọc
            cv2.line(
                face, (i * cell_size, 0), (i * cell_size, face.shape[0]), (0, 0, 0), 1
            )

    def save_current_face(self, frame, height, width):
        """
        Lưu mặt hiện tại vào danh sách các mặt đã lưu
        """
        self.face = frame[
            height // 2 - 150 : height // 2 + 150,
            width // 2 - 150 : width // 2 + 150,
        ]
        self.face = cv2.GaussianBlur(self.face, (5, 5), 0)
        face_name = self.face_order[self.current_face_index]
        self.faces[face_name] = self.face
        self.processed_faces[face_name], self.face_colors[face_name] = (
            self.process_face(self.face)
        )
        for row in self.face_colors[face_name]:
            row.reverse()
            
        print(f"Đã lưu mặt {face_name}.")
        self.print_face_colors(face_name)

    def process_face(self, face):
        """
        Process the face image to extract the colors of the stickers and return processed image and colors
        """
        try:
            hsv_face = cv2.cvtColor(face, cv2.COLOR_BGR2HSV)
        except Exception as e:
            print(f"Lỗi khi chuyển đổi sang HSV: {e}")
            return face, None

        height, width, _ = face.shape
        cell_height, cell_width = height // 3, width // 3

        processed_face = np.zeros_like(face)
        face_colors = []

        for i in range(3):
            row_colors = []
            for j in range(3):
                cell = hsv_face[
                    i * cell_height : (i + 1) * cell_height,
                    j * cell_width : (j + 1) * cell_width,
                ]

                # Lấy mẫu màu
                samples = []
                step = cell_height // 3
                for y in range(step // 2, cell_height, step):
                    for x in range(step // 2, cell_width, step):
                        samples.append(cell[y, x])
                samples = np.array(samples)

                # Xác định màu dựa trên trung vị
                median_color = np.median(samples, axis=0)
                color_name = self.get_color_name(median_color)

                row_colors.append(color_name)
                color_bgr = self.get_bgr_color(color_name)
                cv2.rectangle(
                    processed_face,
                    (j * cell_width, i * cell_height),
                    ((j + 1) * cell_width, (i + 1) * cell_height),
                    color_bgr,
                    -1,
                )
            face_colors.append(row_colors)

        self.draw_grid(processed_face)
        return processed_face, face_colors

    def get_color_name(self, hsv_color):
        """
        Convert HSV color to a human-readable color name
        """
        colors = {
            "white": ([0, 0, 180], [180, 20, 255]),
            "orange": ([0, 50, 70], [9, 255, 255]),
            "red": ([170, 50, 70], [180, 255, 255]),
            "blue": ([90, 50, 70], [128, 255, 255]),
            "green": ([36, 50, 70], [89, 255, 255]),
            "yellow": ([10, 70, 100], [30, 255, 255]),
        }

        detected_color = "unknown"
        min_distance = float("inf")
        for color_name, (lower, upper) in colors.items():
            if self.in_range(hsv_color, lower, upper):
                return color_name

            distance = self.color_distance(hsv_color, lower, upper)
            if distance < min_distance:
                min_distance = distance
                detected_color = color_name

        return detected_color

    def get_bgr_color(self, color_name):
        """
        Return the BGR color corresponding to the color name
        """
        color_dict = {
            "white": (255, 255, 255),
            "orange": (0, 165, 255),
            "red": (0, 0, 255),
            "blue": (255, 0, 0),
            "green": (0, 255, 0),
            "yellow": (0, 255, 255),
            "unknown": (0, 0, 0),
        }
        return color_dict.get(color_name, (0, 0, 0))

    def in_range(self, hsv_color, lower, upper):
        """
        Check if an HSV color is within the given range
        """
        return all(lower[i] <= hsv_color[i] <= upper[i] for i in range(3))

    def color_distance(self, hsv_color, lower, upper):
        """
        Calculate the Euclidean distance between a color and the range midpoint
        """
        midpoint = [(lower[i] + upper[i]) / 2 for i in range(3)]
        return np.linalg.norm(np.array(hsv_color) - np.array(midpoint))

    def print_face_colors(self, face_name):
        """
        Print the colors of a given face
        """
        face_colors = self.face_colors[face_name]
        if face_colors is not None:
            print(f"Màu sắc của mặt {face_name}:")
            for row in face_colors:
                print(" ".join(row))
        else:
            print(f"Mặt {face_name} chưa được xử lý.")



scanner = RubikScanner()
scanner.capture_face()

Đã lưu mặt U.
Màu sắc của mặt U:
white white red
blue blue red
blue blue red
Đã lưu mặt U.
Màu sắc của mặt U:
white white red
blue blue red
blue blue red
Đã lưu mặt U.
Màu sắc của mặt U:
white white red
blue blue red
blue blue red
Đã lưu mặt U.
Màu sắc của mặt U:
white white red
blue blue red
blue blue red
Đã lưu mặt U.
Màu sắc của mặt U:
white white red
blue blue red
blue blue red
Đã lưu mặt D.
Màu sắc của mặt D:
white orange blue
red red blue
red red white
Đã lưu mặt D.
Màu sắc của mặt D:
orange orange blue
red red blue
red red white
Đã lưu mặt D.
Màu sắc của mặt D:
orange orange blue
red red blue
red red white
Đã lưu mặt D.
Màu sắc của mặt D:
orange orange blue
red red blue
red red white
Đã lưu mặt D.
Màu sắc của mặt D:
orange orange blue
red red blue
red red white


In [58]:
for row in scanner.face_colors["U"]:
    row.reverse()

for row in scanner.face_colors["D"]:
    row.reverse()

for row in scanner.face_colors["L"]:
    row.reverse()

for row in scanner.face_colors["R"]:
    row.reverse()

for row in scanner.face_colors["F"]:
    row.reverse()

for row in scanner.face_colors["B"]:
    row.reverse()

In [59]:
rubik_state = scanner.face_colors

# Bản đồ màu
color_map = {
    "white": "U",
    "yellow": "D",
    "green": "L",
    "blue": "R",
    "red": "F",
    "orange": "B",
}


# Hàm chuyển trạng thái Rubik từ từ điển sang chuỗi ký tự
def rubik_to_string(state):
    order = "URFDLB"
    result = []
    for face in order:
        for row in state[face]:
            for color in row:
                result.append(color_map[color])
    return "".join(result)


# Chuyển trạng thái Rubik sang chuỗi
rubik_string = rubik_to_string(rubik_state)
print(f"Chuỗi trạng thái của Rubik: {rubik_string}")

# Thư viện kociemba để giải Rubik
import kociemba


# Hàm đếm số lần xuất hiện của mỗi màu trong chuỗi trạng thái
def count_colors(cube_string):
    colors = "URFDLB"
    count = {color: 0 for color in colors}
    for char in cube_string:
        if char in count:
            count[char] += 1
    return count


color_count = count_colors(rubik_string)
# print(color_count)

# Kiểm tra xem mỗi màu có đúng 9 lần xuất hiện không
for color, count in color_count.items():
    if count != 9:
        print(f"Lỗi: Màu {color} xuất hiện {count} lần, cần xuất hiện đúng 9 lần.")

# Kiểm tra nếu khối Rubik đã giải xong
solved_string = "UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB"
if rubik_string == solved_string:
    print("Khối Rubik đã được giải xong")
else:
    # Kiểm tra và giải mã khối Rubik
    try:
        solution = kociemba.solve(rubik_string)
        print(f"Giải Rubik: {solution}")
    except ValueError as e:
        print(f"Lỗi: {e}")

Chuỗi trạng thái của Rubik: UUFUUFRBFDDRRRRDRRDULDFDURLLFBFDBDDBFFFLLLBLBULLUBBUBR
Giải Rubik: F L' F' R' L F R U' R'


In [9]:
import kociemba


class SolverRubik:
    def __init__(self, state=None):
        self.rubik_state = state
        self.solved_state = "UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB"
        self.color_map = {
            "white": "U",
            "yellow": "D",
            "green": "L",
            "blue": "R",
            "red": "F",
            "orange": "B",
        }

    def rubik_to_string(self, state):
        order = "URFDLB"
        result = []
        for face in order:
            for row in state[face]:
                for color in row:
                    result.append(self.color_map[color])
        return "".join(result)

    def count_colors(self, cube_string):
        colors = "URFDLB"
        count = {color: 0 for color in colors}
        for char in cube_string:
            if char in colors:
                count[char] += 1
        return count

    def solve(self):
        if self.rubik_state is None:
            return "No state provided"
        
        rubik_string = self.rubik_to_string(self.rubik_state)
        color_count = self.count_colors(rubik_string)
        
        if rubik_string == self.solved_state:
            return "Rubik's cube is already solved"
        
        for color, count in color_count.items():
            if count != 9:
                return f"Error: Color {color} appears {count} times, should appear 9 times"
        
        try:
            solution = kociemba.solve(rubik_string)
            return solution
        except ValueError as e:
            return f"Error: {e}"
        


state = {
    "U": [["white", "white", "white"], ["white", "white", "white"], ["white", "white", "white"]],   
    "D": [["yellow", "yellow", "yellow"], ["yellow", "yellow", "yellow"], ["yellow", "yellow", "yellow"]],
    "L": [["green", "green", "green"], ["green", "green", "green"], ["green", "green", "green"]],
    "R": [["blue", "blue", "blue"], ["blue", "blue", "blue"], ["blue", "blue", "blue"]],
    "F": [["red", "red", "red"], ["red", "red", "red"], ["red", "red", "red"]],
    "B": [["orange", "orange", "orange"], ["orange", "orange", "orange"], ["orange", "orange", "orange"]],
}

s = SolverRubik(state)
print(s.solve())

Rubik's cube is already solved


In [1]:
import tkinter as tk

def update_text():
    label.config(text="New Text")
    root.after(2000, clear_text)  # Clear the text after 2 seconds

def clear_text():
    label.config(text="")
    root.after(2000, update_text)  # Update the text again after 2 seconds

root = tk.Tk()
root.title("Text Update Example")

frame = tk.Frame(root)
frame.pack(padx=10, pady=10)

label = tk.Label(frame, text="Original Text")
label.pack()

# Start the text update cycle
root.after(2000, update_text)

root.mainloop()
